In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

!pip install imbalanced-learn
from imblearn.under_sampling import RandomUnderSampler

c:\Users\c678456\AppData\Local\anaconda3\envs\dmeyf\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#2. Constantes.
#a. Generales.z
%run "../constantes.ipynb"

#b. A definir por el usuario.
dataset_con_fe = dataset_file_fe_12_3_DQ

cantidad_meses_train = 12
ventana = 3

mes_train = mes_train_ult_anio
mes_test = mes_test

dataset_output =  dataset_file_fe_12_3_undersampleado_DQ_B2

In [ ]:
#3. Funciones
%run "../funciones.ipynb"

In [ ]:
#4. Lectura de datos.
data = pd.read_parquet(dataset_con_fe)

In [ ]:
#5. Divido entre clases.
data['clase_binaria2'] = 0
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#6. Filtro solamente los datos de entrenamiento.
data = data[data["foto_mes"].isin(mes_train)]

In [ ]:
#7. Calculo las proporciones.
proportions = (
    data.groupby('foto_mes')['clase_binaria2']
    .value_counts()
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

In [ ]:
#8. Undersampleo.
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria2")
    y = group["clase_binaria2"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 5.
    new_proportion = minority_proportion * 5

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

In [ ]:
#9. Nuevas proporciones.
proportions = (
    data.groupby('foto_mes')['clase_binaria2']
    .value_counts()
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

In [ ]:
#10. Guardamos el modelo.
data.to_parquet(dataset_output, index=False)

In [ ]:
#11. Mensaje.
print("Terminó.")